In [0]:

/*
============================================================
Stored Procedure: Load Bronze Layer (Source => Bronze)
============================================================
Script Purpose:
  This stored procedure loads data into the 'bronze' schema from external CSV files.
  It performs the following actions:
    - Truncates the bronze tables before loading the data.
    - Uses the 'read_files()' function to load the data from csv files to bronze tables.

Parameters:
  - None.
  This stored procedure doesn't accept any parameters or return any values

Usage Example:
  -  CALL datawarehouse.bronze.load_bronze();
*/

-- Execute Load_bronze() function 
call datawarehouse.bronze.load_bronze();

-- Build Store Procedure to execute it once while loading the data into Bronze Layer

CREATE OR REPLACE PROCEDURE datawarehouse.bronze.load_bronze() 
LANGUAGE SQL 
SQL SECURITY INVOKER
COMMENT 'Exceuting load methods to load all CSV file into bronze tables'
AS
BEGIN

  DECLARE START_TIME TIMESTAMP;
  DECLARE END_TIME TIMESTAMP;

  SET START_TIME = current_timestamp();
  
  -- Ingest cust_info table
  select printf('============================');

  truncate table datawarehouse.bronze.crm_cust_info;

  INSERT INTO datawarehouse.bronze.crm_cust_info
  SELECT 
    cst_id,
    cst_key,
    cst_firstname,
    cst_lastname,
    cst_marital_status,
    cst_gndr,
    cst_create_date
  from read_files(
    '/Workspace/Users/ammar.sayed@rackspace.com/sql-data-warehouse-project/datasets/source_crm/cust_info.csv',
    format  => 'csv',
    header  => True,
    mode    => 'FAILFAST'
  );

  -- select * from datawarehouse.bronze.crm_cust_info;


  -- Ingest prd_info table
  truncate table datawarehouse.bronze.crm_prd_info;

  INSERT INTO datawarehouse.bronze.crm_prd_info 
  SELECT 
    prd_id,
    prd_key,
    prd_nm,
    prd_cost,
    prd_line,
    prd_start_dt,
    prd_end_dt
  from read_files(
    '/Workspace/Users/ammar.sayed@rackspace.com/sql-data-warehouse-project/datasets/source_crm/prd_info.csv',
    format  => 'csv',
    header  => True,
    mode    => 'FAILFAST'
  );

  -- select * from datawarehouse.bronze.crm_prd_info;




  -- Ingest prd_info table
  truncate table datawarehouse.bronze.crm_sales_details;

  INSERT INTO datawarehouse.bronze.crm_sales_details 
  SELECT 
    sls_ord_num,
    sls_prd_key,
    sls_cust_id,
    DATE_FROM_UNIX_DATE(sls_order_dt) as sls_order_dt,
    DATE_FROM_UNIX_DATE (sls_ship_dt) as sls_ship_dt,
    DATE_FROM_UNIX_DATE (sls_due_dt) as sls_due_dt,
    sls_sales,
    sls_quantity,
    sls_price
  from read_files(
    '/Workspace/Users/ammar.sayed@rackspace.com/sql-data-warehouse-project/datasets/source_crm/sales_details.csv',
    format  => 'csv',
    header  => True,
    mode    => 'FAILFAST'
  );

  -- select * from datawarehouse.bronze.crm_sales_details;



  -- Ingest erp_cust_az12.csv table
  truncate table datawarehouse.bronze.erp_cust_az12;

  INSERT INTO datawarehouse.bronze.erp_cust_az12 
  SELECT 
  CID,
    BDATE,
  GEN
  from read_files(
    '/Workspace/Users/ammar.sayed@rackspace.com/sql-data-warehouse-project/datasets/source_erp/CUST_AZ12.csv',
    format  => 'csv',
    header  => True,
    mode    => 'FAILFAST'
  );

  -- select count(1) from datawarehouse.bronze.erp_cust_az12;



  -- Ingest erp_loc_a101.csv table
  truncate table datawarehouse.bronze.erp_loc_a101;

  INSERT INTO datawarehouse.bronze.erp_loc_a101 
  SELECT 
    CID,
    CNTRY
  from read_files(
    '/Workspace/Users/ammar.sayed@rackspace.com/sql-data-warehouse-project/datasets/source_erp/LOC_A101.csv',
    format  => 'csv',
    header  => True,
    mode    => 'FAILFAST'
  );

  -- select count(1) from datawarehouse.bronze.erp_loc_a101;



  -- Ingest erp_px_cat_g1v2.csv table
  truncate table datawarehouse.bronze.erp_px_cat_g1v2;

  INSERT INTO datawarehouse.bronze.erp_px_cat_g1v2 
  SELECT 
    ID, CAT, SUBCAT, MAINTENANCE
  from read_files(
    '/Workspace/Users/ammar.sayed@rackspace.com/sql-data-warehouse-project/datasets/source_erp/PX_CAT_G1V2.csv',
    format  => 'csv',
    header  => True,
    mode    => 'FAILFAST'
  );


  -- select * from datawarehouse.bronze.erp_px_cat_g1v2;
  SET END_TIME = current_timestamp();

  select printf('execution time: %d seconds', date_diff(second, START_TIME, END_TIME), 'Seconds');


END

